# `import logging`

Going beyond 

```python
logging.basicConfig(level="INFO"); logging.info("Hello")
```

---

**Table of contents**

[TOC]

In [1]:
import logging
import logging.config
import importlib
import sys
import os
import os.path
import time
import random

# use this file when logging to ensure it is cleaned after each cell
LOGFILE = "logging-article.log"

def read_logfile(file: str = LOGFILE) -> str:
    if not os.path.exists(file):
        raise ValueError(f"{file} does not exist")
    with open(file, "r") as fd:
        return fd.read()


def clear_logfile(file: str = LOGFILE) -> None:
    if os.path.exists(file):
        os.unlink(file)


def reload() -> None:
    """Call this at every cell to reset logging config"""
    importlib.reload(logging)
    importlib.reload(logging.config)

    clear_logfile()

# Logging flow

*[Link to docs](https://docs.python.org/3/howto/logging.html#logging-flow)*

![](https://docs.python.org/3/_images/logging_flow.png)

## Objects we need to understand
1. Loggers
2. Handler
3. Formatter
4. Filters
5. Log Record

In [2]:
# How to give a code example of this?

# Loggers
## Naming loggers

Loggers apply the first level of filtering and emit the log record.

To get loggers, we use the `logging.getLogger(__name__)` helper function. 
Using the module  (`__name__`) as logger name is common.

In [3]:
reload()

logger = logging.getLogger("foo")

print(logger, logger.name, type(logger), sep="\n")

<Logger foo (WARNING)>
foo
<class 'logging.Logger'>


## Loggers parents and childs

Loggers are named with herarchical system using dots (`.`).

In [21]:
reload()

logger_parent = logging.getLogger("foo")

logger_child = logging.getLogger("foo.bar")

print(f"The parent of {logger_child=} is {logger_child.parent=}")
print(f"The parent of {logger_child=} is {logger_child.parent=}")  # TODO

The parent of logger_child=<Logger foo.bar (WARNING)> is logger_child.parent=<Logger foo (WARNING)>


Child loggers will propagate (i.e. forward) log records to their parents if:
- The child is enabled for 
- the `propagate` attribute is set to `True`.

**Pseudo code**
```python
## PSEUDO
# logger: Current Logger
# log_record: log record being emited

logger.emit(log_record)
while "logger has parent" and logger.propagate:
    parent_name = logger.name.split(".")[0]
    logger = logging.getLogger(parent_name)

    if logger.disabled or "logger is not enabled for log_record":
        break

    logger.emit(log_record)
```

In [5]:
reload()

logger_parent = logging.getLogger("foo")

logger_child = logging.getLogger("foo.bar")

logger_parent.setLevel(logging.INFO)
logger_child.setLevel(logging.INFO)

# we will go into more depth into handlers and formatters latter
# this will print 
# parent messages with the format ---message----
# child messages with the format /\/\message/\/\

handler_parent = logging.StreamHandler(sys.stdout)
handler_parent.formatter = logging.Formatter("---%(msg)s---")
logger_parent.addHandler(handler_parent)

handler_child = logging.StreamHandler(sys.stdout)
handler_child.formatter = logging.Formatter("/\\/\\%(msg)s/\\/\\")
logger_child.addHandler(handler_child)

# With propagate, the message is emitted by both loggers
logger_child.propagate = True
logger_child.info("Hello")

# Without propagate, the child will never pass the message to the parent
logger_child.propagate = False
logger_child.info("Goodbye")

/\/\Hello/\/\
---Hello---
/\/\Goodbye/\/\


## The Root Logger

The root logger is the parent of all loggers that don't have a dot in their name.

I.e. the root logger is the ancestor of every other logger.

In [6]:
reload()

root_logger = logging.getLogger() # or logging.root
child_of_root = root_logger.getChild("foo")
named_top_level_logger = logging.getLogger("foo")

print("Root logger:", root_logger)
print(f"Q: Is the '{root_logger}' the parent of logger '{named_top_level_logger}'?\nA: {named_top_level_logger.parent is root_logger}")
print(f"Q: Is '{child_of_root=}' the same as '{named_top_level_logger=}'?\nA: {child_of_root is named_top_level_logger}")


Root logger: <RootLogger root (WARNING)>
Q: Is the '<RootLogger root (WARNING)>' the parent of logger '<Logger foo (WARNING)>'?
A: True
Q: Is 'child_of_root=<Logger foo (WARNING)>' the same as 'named_top_level_logger=<Logger foo (WARNING)>'?
A: True


## Loggers: Level

In [7]:
reload()

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

# If the set log level is less or equal than the message
# the log record is emitted 
logger.setLevel(logging.INFO)
logger.info("Hello")

# If the set log level is greater than the message
# the log record is ignored 
logger.setLevel(logging.WARNING)
logger.info("Goodbye")


INFO:foo:Hello


## Loggers: Disabled

In [8]:
reload()

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

logger.disabled = False
logger.info("Hello")

logger.disabled = True
logger.info("Goodbye")

INFO:foo:Hello


## Custom logger classes

You can extend and modify the behaviour of the Logger class.

Some example use cases are:
- Json loggers
- Context loggers
- etc

In [9]:
reload()

class MyNoisyLogger(logging.Logger):
    def makeRecord(self, *args, **kwargs) -> logging.LogRecord:
        print("I'm making a log record")
        return super().makeRecord(*args, *kwargs)

logging.basicConfig(level="INFO")

logging.setLoggerClass(MyNoisyLogger)

logger = logging.getLogger("foo")
logger.info("Hello")

INFO:foo:Hello


I'm making a log record


# Handlers
Handlers act upon log records do perform an specific action.

Some example actions:
- Print to terminal
- Append to file
- Send a HTTP request
- etc

Handlers are also responsible for formatting the log record and transforming them to strings. See [Formatters](#formatters).

The two most common logger examples are:
- `StreamHandler`: prints messages to terminal (defaults to stderr)
- `FileHandler`: writes messages to a file

In [10]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

# without any handlers
logger.info("Without handlers")

# with a StreamHandler
logger.addHandler(logging.StreamHandler(sys.stdout))
print(logger.handlers)
logger.info("With one StreamHandler")

# with another StreamHandler
# the message gets handled by each handler independently
logger.addHandler(logging.StreamHandler(sys.stdout))
print(logger.handlers)
logger.info("With two StreamHandlers")

# Add a FileHandler
logger.addHandler(logging.FileHandler(LOGFILE))
print(logger.handlers)
print("Before emitting log:", repr(read_logfile()))  # before message logfile is empty
logger.info("With two StreamHandlers and a FileHandler")
print("After emitting log:", repr(read_logfile()))  # after message logfile is not empty

[<StreamHandler stdout (NOTSET)>]
With one StreamHandler
[<StreamHandler stdout (NOTSET)>, <StreamHandler stdout (NOTSET)>]
With two StreamHandlers
With two StreamHandlers
[<StreamHandler stdout (NOTSET)>, <StreamHandler stdout (NOTSET)>, <FileHandler /apps/home/oberndof/workspace/python-guild/material/getting-started/logging-article.log (NOTSET)>]
Before emitting log: ''
With two StreamHandlers and a FileHandler
With two StreamHandlers and a FileHandler
After emitting log: 'With two StreamHandlers and a FileHandler\n'


## Handlers: levels and and other attributes
Handlers, like loggers, have a level and [filters](#filters).

Contrary to loggers:
- they are not hierarchical and don't propagate anything
- they do not have a enabled/disabled state

In [11]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

handler: logging.Handler = logging.StreamHandler(sys.stdout)
logger.addHandler(handler)

handler.setLevel(logging.INFO)
print(logger.handlers)
logger.info("Hello")

handler.setLevel(logging.WARNING)
print(logger.handlers)
logger.info("Hello")  # handler still exists but ignores the record

[<StreamHandler stdout (INFO)>]
Hello
[<StreamHandler stdout (WARNING)>]


## Useful Handlers

*[Link to docs](https://docs.python.org/3/howto/logging.html#useful-handlers)*

Python, with its philosophy of "batteries included", has a lot of useful handlers already included in the `logging.handlers` module. Here are some of them:

- `RotatingFileHandler` instances send messages to disk files, with support for maximum log file sizes and log file rotation.
- `TimedRotatingFileHandler` instances send messages to disk files, rotating the log file at certain timed intervals.
- `SocketHandler` instances send messages to TCP/IP sockets.
- `DatagramHandler` instances send messages to UDP sockets.
- `SMTPHandler` instances send messages to a designated email address.
- `SysLogHandler` instances send messages to a Unix syslog daemon, possibly on a remote machine.
- `HTTPHandler` instances send messages to an HTTP server using either GET or POST semantics.
- `QueueHandler` instances send messages to a queue, such as those implemented in the queue or multiprocessing modules.
- `NullHandler`: See [this section  of the notebook](#logging-in-libraries-nullhandler-and-lastresort).

In [12]:
# should we provide an example of this?

# LogRecords
The objects communicated between loggers, handlers and other classes are not string messages, they are of type `LogRecord`.

In [13]:
# lets take a look to the constructor
log_record = logging.LogRecord(
    name="name of the logger tat created the record",
    level=logging.INFO,
    pathname="/path/to/file/that/logged",
    lineno=17,
    msg="the message before formatting args: %s",
    args=("args to format msg",),
    exc_info=None,
    func=None,
    sinfo=None,
)

# and some of its methods
print("LogRecord is:", repr(log_record))
print("The formatted message is:", log_record.getMessage())

LogRecord is: <LogRecord: name of the logger tat created the record, 20, /path/to/file/that/logged, 17, "the message before formatting args: %s">
The formatted message is: the message before formatting args: args to format msg


# Formatters
[Most] Handlers have a formatter. It determines how to convert the LogRecord into a string when emitting the message.

In [14]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

handler: logging.Handler = logging.StreamHandler(sys.stdout)
logger.addHandler(handler)
handler.setFormatter(logging.Formatter("%(message)s"))
logger.info("Without much formatting")
handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(msg)s"))
logger.info("With formatting")
handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(msg)s", datefmt="%Y%m%dT%H:%M"))
logger.info("With message and date formatting")

Without much formatting
2022-11-15 14:59:18,510 - INFO - With formatting
20221115T14:59 - INFO - With message and date formatting


## Formatter attributes
You can pass any LogRecord attribute as keys in your string-format:

| Attribute | Format | Description |
|-----------|--------|-------------|
| asctime | `%(asctime)s` | Human-readable time when the `LogRecord` was created.  By default this is of the form ‘2003-07-08 16:49:45,896’ |
| created | `%(created)f` | Timestamp of `asctime`  |
| filename | `%(filename)s` | Filename portion of `pathname`. |
| funcName | `%(funcName)s` | Name of function containing the logging call. |
| levelname | `%(levelname)s` | Text logging level for the message. |
| levelno | `%(levelno)s` | [Numeric logging level](#numeric-log-levels) for the message. |
| lineno | `%(lineno)d` | Source line number where the logging call was issued  (if available). |
| message | `%(message)s` | The logged message, computed as `msg % args`. This is set when `Formatter.format()` is invoked. |
| module | `%(module)s` | Module (name portion of `filename`). |
| msecs | `%(msecs)d` | Millisecond portion of the time when the `LogRecord` was created. |
| name | `%(name)s` | Name of the logger used to log the call. |
| pathname | `%(pathname)s` | Full pathname of the source file where the logging call was issued (if available). |
| process | `%(process)d` | Process ID (if available). |
| processName | `%(processName)s` | Process name (if available). |
| relativeCreated | `%(relativeCreated)d` | Time in milliseconds when the LogRecord was created, relative to the time the logging module was loaded. |
| thread | `%(thread)d` | Thread ID (if available). |
| threadName | `%(threadName)s` | Thread name (if available). |

PS: Do not confuse `msg` and `message`. You are probably need `%(message)s` as `msg` is before formatting the args.


In [15]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
logger.addHandler(handler)
handler.setFormatter(logging.Formatter("[%(asctime)s][%(levelname)s][%(name)s:%(lineno)s][%(processName)s:%(threadName)s] %(message)s"))
logger.info("This is a %s example of formatting", "good")

[2022-11-15 14:59:18,596][INFO][foo:9][MainProcess:MainThread] This is a good example of formatting


## Formatter styles
It is possible to use f-style formatting or even env var expansion, with the `style` argument of the formatter.

However, the messages emitted by the logger style have to use the `'%s'` style.

In [16]:
reload()

logger = logging.getLogger("foo")
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
logger.addHandler(handler)

handler.setFormatter(logging.Formatter("[%(levelname)s] %(message)s", style="%"))
logger.info("Logging with style %r", "%")

handler.setFormatter(logging.Formatter("[$levelname] $message", style="$"))
logger.info("Logging with style %r", "$")

handler.setFormatter(logging.Formatter("[{levelname}] {message}", style="{"))
logger.info("Logging with style %r", "{")
logger.info("Logging with style {}", "{") # this will create an error

[INFO] Logging with style '%'
[INFO] Logging with style '$'
[INFO] Logging with style '{'


--- Logging error ---
Traceback (most recent call last):
  File "/opt/rh/rh-python38/root/usr/lib64/python3.8/logging/__init__.py", line 1081, in emit
    msg = self.format(record)
  File "/opt/rh/rh-python38/root/usr/lib64/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/opt/rh/rh-python38/root/usr/lib64/python3.8/logging/__init__.py", line 664, in format
    record.message = record.getMessage()
  File "/opt/rh/rh-python38/root/usr/lib64/python3.8/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/rh/rh-python38/root/usr/lib64/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/rh/rh-python38/root/usr/lib64/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/lxhome/oberndof/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <mo

## How to choose a logging format

Ask around, the best logging format is the one that is already used by other projects.

If your team does not already have a standardized format, choose a format:
- Contains enough information without being too much. E.g. If you have `name` as `__name__` then you don't need `module` or `pathname`.
- Choose a format that is both human and machine readable. Your logs are usually parsed by both live log collections systems and log analytics, but they are also read by humans.

# Filters

Filters are functions or classes that are called upon a LogRecord to determine if it should be filtered.

In [17]:
reload()

def filter_ignore_me(record: logging.LogRecord) -> bool:
    """True means the message should be preserved"""
    return "IGNORED" not in record.msg

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")
logger.info("Hello")
logger.info("This message should be IGNORED but there is no filter")

logger.addFilter(filter_ignore_me)
logger.info("This message should also be IGNORED")
logger.info("Goodbye")

INFO:foo:Hello
INFO:foo:This message should be IGNORED but there is no filter
INFO:foo:Goodbye


In [18]:
reload()

class MyFilter(logging.Filter):
    """A Filter can also be an instance of logging.Filter"""

    def filter(self, record: logging.LogRecord) -> bool:
        """A filter may also modify inplace the Log Record"""
        if "IGNORED" in record.msg:
            return False
        if "IMPORTANT" in record.msg:
            # I wouldn't do this in real life
            record.levelname = "WARNING"
            record.levelno = 30
            record.msg = record.msg.replace("IMPORTANT", "").strip()
        return True

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

logger.addFilter(MyFilter())
logger.info("Hello")
logger.info("This message should be IGNORED")
logger.info("IMPORTANT Goodbye")

INFO:foo:Hello


## Filter in both Loggers and Handlers

# Optimizing logging

## Late formatting

## Collecting less information

In [19]:
reload()

logging.logThreads = False
logging.logProcesses = False
logging.logMultiprocessing = False

# Check before logging

In [20]:
reload()

logging.basicConfig(level="INFO")

logger = logging.getLogger("foo")

def expensive_func():
    time.sleep(3)
    return random.random()

if logger.isEnabledFor(logging.DEBUG):
    logger.debug('Message with %s', expensive_func())

# Tips and FAQ
1. Numeric log levels
2. Why late formatting
3. Logging in libraries: NullHandler and LastResort.
4. `extra`
5. Logging exceptions
6. The stack_info var ?

## Numeric Log levels

In [22]:
logging.INFO

20

## Logging in libraries: NullHandler and LastResort

## The `extra` argument to handlers

## Logging exception tracebacks